<a href="https://colab.research.google.com/github/leetaebeom/python_tb/blob/main/13%EC%A3%BC%EC%B0%A8_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# 예시 ISBN 리스트 (테스트용으로 3개만)
isbn_list = ["9788960777330", "9788968481475", "9788998139766"]

# 쪽수와 저자 정보 함께 추출하는 함수
def get_page_author(isbn):
    url = f"https://search.kyobobook.co.kr/web/search?vPstrKeyWord={isbn}"
    headers = {
        "User-Agent": "Mozilla/5.0"
    }

    try:
        res = requests.get(url, headers=headers)
        soup = BeautifulSoup(res.text, 'html.parser')

        # 1. 도서 상세 페이지 링크 추출
        link_tag = soup.select_one("div.title > a")
        if not link_tag:
            return None, None
        detail_link = "https://search.kyobobook.co.kr" + link_tag.get("href")

        # 2. 상세 페이지 접근
        detail_res = requests.get(detail_link, headers=headers)
        detail_soup = BeautifulSoup(detail_res.text, 'html.parser')

        # 3. 쪽수와 저자 추출
        page = None
        author = None
        info_list = detail_soup.select("div.publish-info > ul > li")

        for li in info_list:
            text = li.text.strip()
            if "쪽수" in text:
                page = text.split(":")[-1].strip().replace("쪽", "")
            if "저자" in text or "글쓴이" in text:
                author = li.select_one("a").text.strip()

        return page, author
    except:
        return None, None

# 결과 저장
results = []

for isbn in isbn_list:
    page, author = get_page_author(isbn)
    results.append({
        "ISBN": isbn,
        "Page": page,
        "Author": author
    })
    time.sleep(1)  # 과도한 요청 방지

# 데이터프레임 생성
df = pd.DataFrame(results)
df


In [ ]:
def get_book_info(isbn):
    url = f"https://search.kyobobook.co.kr/web/search?vPstrKeyWord={isbn}"
    headers = {"User-Agent": "Mozilla/5.0"}

    try:
        res = requests.get(url, headers=headers)
        soup = BeautifulSoup(res.text, 'html.parser')
        link_tag = soup.select_one("div.title > a")
        if not link_tag:
            return None, None, None
        detail_link = "https://search.kyobobook.co.kr" + link_tag.get("href")

        detail_res = requests.get(detail_link, headers=headers)
        detail_soup = BeautifulSoup(detail_res.text, 'html.parser')

        page = size = author = None
        info_list = detail_soup.select("div.publish-info > ul > li")

        for li in info_list:
            text = li.text.strip()
            if "쪽수" in text:
                page = text.split(":")[-1].strip().replace("쪽", "")
            elif "크기" in text:
                size = text.split(":")[-1].strip()
            elif "저자" in text:
                author = li.select_one("a").text.strip()

        return page, size, author
    except:
        return None, None, None
